In [1]:
from __future__ import print_function, division
from builtins import range
import numpy as np

## make Gridworld

In [2]:

ACTION_SPACE = ('U', 'D', 'L', 'R' ,'U2')


class Grid:  # Environment

    def __init__(self,rows,cols,start):
        self.rows = rows
        self.cols = cols
        self.i = start[0]
        self.j = start[1]

    def set(self, rewards, actions):

        self.rewards = rewards
        self.actions = actions

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions

    def get_next_state(self, s, a):

    # this answers: where would I end up if I perform action 'a' in state 's'?

        (i, j) = (s[0], s[1])

    # if this action moves you somewhere else, then it will be in this dictionary

        if a in self.actions[(i, j)]:
            if a == 'U':
                i -= 1
            elif a == 'D':
                i += 1
            elif a == 'R':
                j += 1
            elif a == 'L':
                j -= 1
            elif a == 'U2':
                i += 2
        return (i, j)


    def all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())





## Define Grid 4*5

In [6]:
def standard_grid():

  # .  x  x  1 .
  # .  x   -1 .
  # s  .  .  . .
  # .  .  .  . .

    grid = Grid(4, 5, (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0, 0): ('D'),
        (0, 4): ('L'),
        (1, 0): ('U', 'D'),
        (1, 2): ('D', 'R'),
        (1, 4): ('U', 'L', 'D'),
        (2, 0): ('U', 'R', 'D'),
        (2, 1): ('L', 'R', 'D'),
        (2, 2): ('L', 'R', 'D',  'U'),
        (2, 3): ('L', 'R', 'D',  'U'),
        (2, 4): ('L', 'D',  'U'),
        (3, 0): ('R', 'U'),
        (3, 1): ('L', 'R',  'U'),
        (3, 2): ('L', 'R',  'U'),
        (3, 3): ('L', 'R',  'U'),
        (3, 4): ('L', 'U'),
        }
    grid.set(rewards, actions)
    return grid

## Display Grid

In [7]:
def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")  
        print("")


def print_policy(P, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            a = P.get((i, j), " ")
            print("  %s  |" % a, end="")
        print("")


## evaluate fix policy and find optimal value functions

In [23]:
grid = standard_grid()

### fixed policy ###
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'D',
    (0, 1): 'X',
    (0, 2): 'X',
    (1, 2): 'D',
    (2, 1): 'R',
    (2, 2): 'U',
    (2, 3): 'L',
    (1, 1): 'X',
    (1, 3): 'H',
    (3, 0): 'R',
    (3, 1): 'R',
    (3, 2): 'R',
    (3, 3): 'R',
    (1, 4): 'U',
    (2, 4): 'U',
    (3, 4): 'U',
    (0, 4): 'D',
    (3, 4): 'U',
    (0, 3): 'G',
}
print_policy(policy, grid)


    

---------------------------
  D  |  X  |  X  |  G  |  D  |
---------------------------
  U  |  X  |  D  |  H  |  U  |
---------------------------
  U  |  R  |  U  |  L  |  U  |
---------------------------
  R  |  R  |  R  |  R  |  U  |


In [24]:
# initialize V(s) = 0
V = {}
for s in grid.all_states():
    V[s] = 0

print_values(V, grid)

---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|


## Reward and Transition probability

In [25]:
transition_probs = {}
rewards = {}

for row in range(grid.rows):
    for col in range(grid.cols):
        state = (row, col)
        if not grid.is_terminal(state):
            for action in ACTION_SPACE:
                next_state = grid.get_next_state(state, action)
                transition_probs[(state, action, next_state)] = 1
                if next_state in grid.rewards:
                    rewards[(state, action, next_state)] = grid.rewards[next_state]


print(rewards)
print("---------------")
print(transition_probs)

{((0, 4), 'L', (0, 3)): 1, ((1, 2), 'R', (1, 3)): -1, ((1, 4), 'L', (1, 3)): -1, ((2, 3), 'U', (1, 3)): -1}
---------------
{((0, 0), 'U', (0, 0)): 1, ((0, 0), 'D', (1, 0)): 1, ((0, 0), 'L', (0, 0)): 1, ((0, 0), 'R', (0, 0)): 1, ((0, 0), 'U2', (0, 0)): 1, ((0, 4), 'U', (0, 4)): 1, ((0, 4), 'D', (0, 4)): 1, ((0, 4), 'L', (0, 3)): 1, ((0, 4), 'R', (0, 4)): 1, ((0, 4), 'U2', (0, 4)): 1, ((1, 0), 'U', (0, 0)): 1, ((1, 0), 'D', (2, 0)): 1, ((1, 0), 'L', (1, 0)): 1, ((1, 0), 'R', (1, 0)): 1, ((1, 0), 'U2', (1, 0)): 1, ((1, 2), 'U', (1, 2)): 1, ((1, 2), 'D', (2, 2)): 1, ((1, 2), 'L', (1, 2)): 1, ((1, 2), 'R', (1, 3)): 1, ((1, 2), 'U2', (1, 2)): 1, ((1, 4), 'U', (0, 4)): 1, ((1, 4), 'D', (2, 4)): 1, ((1, 4), 'L', (1, 3)): 1, ((1, 4), 'R', (1, 4)): 1, ((1, 4), 'U2', (1, 4)): 1, ((2, 0), 'U', (1, 0)): 1, ((2, 0), 'D', (3, 0)): 1, ((2, 0), 'L', (2, 0)): 1, ((2, 0), 'R', (2, 1)): 1, ((2, 0), 'U2', (2, 0)): 1, ((2, 1), 'U', (2, 1)): 1, ((2, 1), 'D', (3, 1)): 1, ((2, 1), 'L', (2, 0)): 1, ((2, 1), 'R

## Policy evaluation using bootstrapping

In [26]:
iteration = 0
while True:
    for state in grid.all_states():
        if not grid.is_terminal(state):
            new_v = 0
            for action in ACTION_SPACE:
                for next_state in grid.all_states():
                    #deteministic actions:
                    action_prob = 1 if policy.get(state) == action else 0
                
                    r = rewards.get((state, action, next_state), -0.91)
                    new_v += action_prob * transition_probs.get((state, action, next_state), 0) * (r + 0.9 * V[next_state])
                    
                    
            V[state] = new_v
            
    
    
    iteration = iteration+1
    print_values(V, grid)
    print("\n")
    if iteration == 25:
        break
    
            

---------------------------
-0.91| 0.00| 0.00| 0.00|-0.91|
---------------------------
-1.73| 0.00|-0.91| 0.00|-1.73|
---------------------------
-0.91|-0.91|-1.73|-0.91|-0.91|
---------------------------
-1.73|-0.91|-3.13|-2.47|-1.73|


---------------------------
-2.47| 0.00| 0.00| 0.00|-1.73|
---------------------------
-3.13| 0.00|-2.47| 0.00|-2.47|
---------------------------
-2.47|-2.47|-3.13|-2.47|-2.47|
---------------------------
-4.26|-3.73|-4.26|-3.73|-3.13|


---------------------------
-3.73| 0.00| 0.00| 0.00|-2.47|
---------------------------
-4.26| 0.00|-3.73| 0.00|-3.13|
---------------------------
-3.73|-3.73|-4.26|-3.73|-3.13|
---------------------------
-5.18|-4.75|-4.75|-4.26|-3.73|


---------------------------
-4.75| 0.00| 0.00| 0.00|-3.13|
---------------------------
-5.18| 0.00|-4.75| 0.00|-3.73|
---------------------------
-4.75|-4.75|-5.18|-4.75|-3.73|
---------------------------
-5.57|-5.18|-5.18|-4.75|-4.26|


---------------------------
-5.57| 0.00| 0.00| 0